#### 거리측정 함수 정의1

In [ ]:
def x(distance):
    d1 = 30
    y1 = 82
    d2 = 50
    y2 = 47
#   d3 = 40
    y3 = 62
    
    dis = ((d2-d1)/(y2-y1))*(distance-y1)+d1
    d1 = 
    return round(dis,1)

x(62)

In [ ]:
def x2(distance):
    d1 = 30
    y1 = 82
    d2 = 50
    y2 = 47
    d3 = 40
    y3 = 62
    
    dis = (((d1*y1)+(d2*y2)+(d3*y3))/3/distance)
    return round(dis,1)

x2(62)

## 거리측정 + 눈 깜빡임 통합모델

In [ ]:
## 혼합본 (거리 + 깜빡임)

import cv2, dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import tensorflow as tf
import time

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2018_12_17_22_58_35.h5')
model.summary()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config) 

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

def dis(distance):
    d1 = 30
    y1 = 82
    d2 = 50
    y2 = 47
#   d3 = 40
    y3 = 62
    
    dis = ((d2-d1)/(y2-y1))*(distance-y1)+d1
    return round(dis,1)

# main
cap = cv2.VideoCapture(0)

sum = 0
co = 0 
c = 0
cnt = 0
st = 0
stat = [1,1]

while cap.isOpened():
    ret, img_ori = cap.read()

    if not ret:
        break

    img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.9, fy=0.9)
    
    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)
        
        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        
        state_l = '%.1f' if pred_l > 0.1 else '%.1f'
        state_r = '%.1f' if pred_r > 0.1 else '%.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r
        
#         print(stat[0]==1)
#         print(type(state_l))
#         print(state_l == '0.0')
        
        if (stat[0]==1) and (state_l == '0.0') and (stat[1]==1) and (state_r == '0.0') :
            stat[0] = 0
            stat[1] = 0
            cnt += 1
        
        if (state_l == '1.0') :
            stat[0] = 1
            
        if (state_r == '1.0') :
            stat[1] = 1
        
#         print(state_l,state_r)
#         print('st : ',st)
#         print(stat,cnt)
        
        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

        cv2.putText(img, "state : {}".format(cnt), (0,80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (71,100,62), 2)


        # 눈 사이 거리 측정
        
        centerX_l = eye_rect_l[0] + (eye_rect_l[2] - eye_rect_l[0])/2
        centerY_l = eye_rect_l[1] + (eye_rect_l[3] - eye_rect_l[1])/2
        centerX_r = eye_rect_r[0] + (eye_rect_r[2] - eye_rect_r[0])/2 
        centerY_r = eye_rect_r[1] + (eye_rect_r[3] - eye_rect_r[1])/2

        distance = np.sqrt((centerX_r- centerX_l)**2 + (centerY_r- centerY_l)**2)
        #print(distance)
        
        #c = x2(distance)
        #print(c)
        
        sum = sum + distance
        co = co + 1

        if(co > 10):
            avg = sum / co
#             print("dis = {}".format(avg))
            
            c = dis(distance)
                    
            sum = 0
            co = 0 
        # 눈 사이 거리값 텍스트 표시
        cv2.putText(img, "distance : " + str(c) +" cm", (0,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (71,200,62), 2)
        
    cv2.imshow('reqsult', img)
    if cv2.waitKey(1) == ord('q'):
        cap.release()    
        cv2.destroyAllWindows()
        break

## 플라스크

In [1]:
import cv2, dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import tensorflow as tf
import json
import matplotlib.pyplot as plt
%matplotlib inline

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2018_12_17_22_58_35.h5')
model.summary()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config) 

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

def dis(distance):
    d1 = 30
    y1 = 82
    d2 = 50
    y2 = 47
#   d3 = 40
    y3 = 62
    
    dis = ((d2-d1)/(y2-y1))*(distance-y1)+d1
    return round(dis,1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26, 34, 1)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 34, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 17, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 17, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 4, 128)         0   

In [ ]:
from flask import Flask, redirect, request, Response, jsonify
import base64
import io
from PIL import Image
import cv2, dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import tensorflow as tf
import json

sum1 = 0
co = 0 
c = 0
cnt = 0
spec = {}
stat=[1,1]

app = Flask(__name__)

@app.route("/re", methods=['POST']) 
def re():
    print(request.method)
    sum1 = 0
    co = 0 
    c = 0
    cnt = 0
    if request.method == 'POST':
        one_data = request.form['img']
        imgdata = base64.b64decode(one_data)
        image = Image.open(io.BytesIO(imgdata))
        img90 = image.rotate(90)
        
        #안드로이드에서 전송된 이미지 확인
        
        pil_image = img90.convert('RGB') 
        open_cv_image = np.array(pil_image) 
        # Convert RGB to BGR 
        open_cv_image = open_cv_image[:, :, ::-1].copy() 

        img_ori = cv2.resize(open_cv_image, dsize=(0, 0), fx=0.9, fy=0.9)
        
        img = img_ori.copy()
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         img90 = cv2.rotate(img_gray, cv2.ROTATE_90_CLOCKWISE)
        print('컨버터 완료')

        display(plt.imshow(img))
        display(plt.show())
        
        faces = detector(img)
        
        print('얼굴찾기 전')
        spec = {'dist':c}
        json_val=json.dumps(spec)
        
        for face in faces:
            shapes = predictor(img_gray, face)
            shapes = face_utils.shape_to_np(shapes)
            print('얼굴 찾음')

            eye_img_l, eye_rect_l = crop_eye(img_gray, eye_points=shapes[36:42])
            eye_img_r, eye_rect_r = crop_eye(img_gray, eye_points=shapes[42:48])
            print('눈 찾음')
            
            eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
            eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
            eye_img_r = cv2.flip(eye_img_r, flipCode=1)
            print('사이즈 조정')

            eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
            eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

            pred_l = model.predict(eye_input_l)
            pred_r = model.predict(eye_input_r)
            
            # 눈 사이 거리 측정

            centerX_l = eye_rect_l[0] + (eye_rect_l[2] - eye_rect_l[0])/2
            centerY_l = eye_rect_l[1] + (eye_rect_l[3] - eye_rect_l[1])/2
            centerX_r = eye_rect_r[0] + (eye_rect_r[2] - eye_rect_r[0])/2 
            centerY_r = eye_rect_r[1] + (eye_rect_r[3] - eye_rect_r[1])/2

            distance = np.sqrt((centerX_r- centerX_l)**2 + (centerY_r- centerY_l)**2)
            c = dis(distance)
            
            state_l = '%.1f' if pred_l > 0.1 else '%.1f'
            state_r = '%.1f' if pred_r > 0.1 else '%.1f'

            state_l = state_l % pred_l
            state_r = state_r % pred_r

    #         print(stat[0]==1)
    #         print(type(state_l))
    #         print(state_l == '0.0')

    #오리지날 깜빡임 횟수체크
#             if (stat[0]==1) and (state_l == '0.0') and (stat[1]==1) and (state_r == '0.0') :
#                 stat[0] = 0
#                 stat[1] = 0
#                 cnt += 1

#             if (state_l == '1.0') :
#                 stat[0] = 1

#             if (state_r == '1.0') :
#                 stat[1] = 1
            if (stat[0]==1 and stat[1]==1):
                if (state_l == '0.0') and (state_r == '0.0'):
                    stat[0] = 0
                    stat[1] = 0
                    cnt += 1

            if (state_l == '1.0') :
                stat[0] = 1

            if (state_r == '1.0') :
                stat[1] = 1
        
#             cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
#             cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)
            
            print(state_l,"-",state_r)
#             print('메소드 적용 후 :',c,"\,",cnt)
#             print(type(c))
            spec = {'dist':float(c), 'blink':cnt}
            print('딕션 적용 후 :',spec)
            json_val=json.dumps(spec)
            print('제이슨 적용 후 :',json_val)
            
            print('for문 실행 완료')
#             cv2.putText(img, "distance : " + str(json_val[0]) +" cm", (0,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (71,200,62), 2)
    
    resp = Response("{}".format(json_val), mimetype='text/plain')
    resp.headers['Access-Control-Allow-Origin'] = '*'
    resp.headers['Access-Control-Allow-Methods'] = 'POST'
    resp.headers['Access-Control-Allow-Headers'] = 'Origin'
    return resp

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple('192.168.56.1', 9000, app)

 * Running on http://192.168.56.1:9000/ (Press CTRL+C to quit)
